In [1]:
!pip install  dgl -f https://data.dgl.ai/wheels/torch-2.4/cu124/repo.html

Looking in links: https://data.dgl.ai/wheels/torch-2.4/cu124/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.

In [9]:
!gdown 1szBOuGckL7gYw5tDPeFjNUpE79DoBF0r

Downloading...
From (original): https://drive.google.com/uc?id=1szBOuGckL7gYw5tDPeFjNUpE79DoBF0r
From (redirected): https://drive.google.com/uc?id=1szBOuGckL7gYw5tDPeFjNUpE79DoBF0r&confirm=t&uuid=5264057b-61be-450f-9c0e-28074ff79a3a
To: /content/train_visual_features.zip
100% 7.94G/7.94G [01:27<00:00, 90.6MB/s]


In [10]:
!gdown 1SFgOGGB8a0JxTaa7phzYnq6K6JyWDGbf

Downloading...
From: https://drive.google.com/uc?id=1SFgOGGB8a0JxTaa7phzYnq6K6JyWDGbf
To: /content/train_data_graphs.bin
100% 86.3M/86.3M [00:01<00:00, 65.1MB/s]


In [11]:
!unzip  -q train_visual_features.zip

In [12]:
from torch.utils.data import Dataset, DataLoader
import pickle
import dgl
import os
import torch
import numpy as np
class GraphDataset(Dataset):
    def __init__(self, graph_file, feature_dir):
        super().__init__()
        self.graphs,_ = dgl.load_graphs(graph_file)

        self.feature_dir = feature_dir

    def __len__(self):
        return len(self.graphs)

    def load_feat(self,nodes):
        tensors = []
        for idx,node in enumerate(nodes):
            try:
                tensors.append(torch.load(f"{self.feature_dir}/{node}.pt",map_location=torch.device("cpu"),weights_only=False).unsqueeze(0))
            except:
                tensors.append(torch.zeros((1,1536)))
        return torch.cat(tensors,dim=0)

    def __getitem__(self, index):
        g = self.graphs[index]
        nodes = g.ndata['obj_id']
        feats = self.load_feat(nodes)
        return g, feats


In [13]:
import torch
import torch.nn as nn
import dgl
import torch.nn.functional as F
class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)

    def apply_edges(self, edges):
        """
        Computes a scalar score for each edge of the given graph.

        Parameters
        ----------
        edges :
            Has three members ``src``, ``dst`` and ``data``, each of
            which is a dictionary representing the features of the
            source nodes, the destination nodes, and the edges
            themselves.

        Returns
        -------
        dict
            A dictionary of new edge features.
        """
        h = torch.cat([edges.src['h'], edges.dst['h']], 1)
        return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']

In [14]:
dataset = GraphDataset('train_data_graphs.bin','train_visual_features')

In [15]:
from torch.utils.data import random_split
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
train_dataset, val_dataset = random_split(dataset,[0.9,0.1])

In [16]:
train_dataloader = DataLoader(train_dataset, batch_size=1, collate_fn=lambda batch: batch[0])
val_dataloader = DataLoader(val_dataset, batch_size=1, collate_fn=lambda batch: batch[0])

In [17]:
model = MLPPredictor(1536)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
from torch import optim
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [18]:
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
best_auc = 0.0
for epoch in tqdm(range(10)):
    total_loss = 0.0
    model.train()
    for g, feats in tqdm(train_dataloader):
        scores = model(g.to(device),feats.to(device))
        labels = g.edata['labels']
        loss = F.binary_cross_entropy_with_logits(scores, labels.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss+=loss.item()
        g.to(torch.device('cpu'))
        feats.to(torch.device('cpu'))
        torch.cuda.empty_cache()
    total_loss/=len(train_dataloader)
    print(f'Epoch: {epoch} Train Loss: {total_loss}')
    model.eval()
    total_loss = 0.0
    auc = 0.0
    with torch.no_grad():
        for g, feats in val_dataloader:
            scores = model(g.to(device),feats.to(device))
            labels = g.edata['labels']
            loss = F.binary_cross_entropy_with_logits(scores, labels.to(device))
            total_loss+=loss.item()
            auc+= roc_auc_score(labels.detach().numpy(), scores.detach().numpy())
        total_loss/=len(val_dataloader)
        auc/=len(val_dataloader)
    if auc > best_auc:
        auc = best_auc
        torch.save(model.state_dict(),"predictor.pth")
    print(f'Epoch: {epoch} Val Loss: {total_loss} AUC: {auc}')


  0%|          | 0/10 [00:09<?, ?it/s]


RuntimeError: torch.cat(): expected a non-empty list of Tensors

In [30]:
dataset.graphs[82]

Graph(num_nodes=0, num_edges=0,
      ndata_schemes={'obj_id': Scheme(shape=(), dtype=torch.float32)}
      edata_schemes={'labels': Scheme(shape=(), dtype=torch.float32)})

In [23]:
!find train_visual_features/ -type f -name "140*.pt"

train_visual_features/14017.pt
train_visual_features/14019.pt
train_visual_features/14021.pt
train_visual_features/14020.pt
train_visual_features/14009.pt
train_visual_features/14015.pt
train_visual_features/14011.pt
train_visual_features/14012.pt
train_visual_features/14010.pt
train_visual_features/14008.pt
train_visual_features/14005.pt
train_visual_features/14004.pt
train_visual_features/14013.pt
train_visual_features/14006.pt
train_visual_features/14018.pt
train_visual_features/14000.pt
train_visual_features/14016.pt
train_visual_features/14007.pt
train_visual_features/14014.pt
train_visual_features/14003.pt
train_visual_features/14002.pt
train_visual_features/140.pt
train_visual_features/1400.pt
train_visual_features/1402.pt
train_visual_features/1401.pt
train_visual_features/14035.pt
train_visual_features/14031.pt
train_visual_features/14034.pt
train_visual_features/14029.pt
train_visual_features/14023.pt
train_visual_features/14033.pt
train_visual_features/14030.pt
train_visual_